In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

In [4]:
import torch
from transformers.tokenization_utils_base import BatchEncoding
from transformers import GPT2Config, AutoTokenizer, AutoModelForCausalLM

In [7]:
class Data_gen(torch.utils.data.Dataset):
    def __init__(self, text_inputs: BatchEncoding, target: BatchEncoding): 
        self.text_inputs = text_inputs
        self.target = target
    def __len__(self):
        return len(self.target)        
    def __getitem__(self, index):
        return {'input_num': torch.LongTensor(self.text_inputs[index].ids),'input_attention': torch.LongTensor(self.text_inputs[index].attention_mask),'target_ids': torch.LongTensor(self.labels[index].ids)}


def vocab_genarator(text_data, sequence_length):
    S = []
    W = []
    vocab = set()
    text_data = text_data.replace('\n',' ')
    splitted_text = text_data.split()
    for x in range(0,len(splitted_text) - 2*sequence_length):
        sentence = ' '.join(splitted_text[x:x+sequence_length])
        word = ' '.join(splitted_text[x+sequence_length: x+2*sequence_length])
        S.append(sentence)
        W.append(word)
        vocab.update(splitted_text[x:x+sequence_length])
        x = x+1
        vocab.update(word)        
    return S, W, vocab

def generate_song(model, seed_string):
    song = seed_string
    current_seed = seed_string
    for x in range(20):
        song_tokens = lang_tokenizer.encode(current_seed, return_tensors="pt").to(torch.device('cpu'))
        song_prediction = model.generate(song_tokens, min_length=10, max_length=30, do_sample=True, repetition_penalty=1.5, temperature=1.5)
        song = song+ f'({x}): \t{lang_tokenizer.decode(song_prediction[0])[len(current_seed):]}\n'
        current_seed = lang_tokenizer.decode(song_prediction[0])[len(current_seed):]

if __name__ == '__main__':# This fuction iDistilGPT2 English language model
    S, W, vocab = vocab_genarator(open('/content/drive/MyDrive/Assignment3/song.txt').read(), 25)
    lang_tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
    lang_tokenizer.pad_token = lang_tokenizer.eos_token
    S_enc=lang_tokenizer(S, truncation=True, padding=True)
    W_enc=lang_tokenizer(W, truncation=True, padding=True)
    data_to_loader = Data_gen(S_enc, W_enc)
    model = AutoModelForCausalLM.from_pretrained('distilgpt2')
    model.to(torch.device('cpu'))
    adams_optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    train_data_loader = torch.utils.data.DataLoader(data_to_loader, batch_size=100, shuffle=False)

In [ ]:
    for curr_epoch in range(40):
    #Train across 40 epochs  
        lang_tokens = lang_tokenizer.encode("Let the darkness lead us into the light", return_tensors="pt").to(torch.device('cpu'))
        song_prediction = model.generate(lang_tokens, min_length=10, max_length=30, do_sample=True, repetition_penalty=1.5, temperature=1.5)
        for i, batch in enumerate(train_data_loader):
            adams_optimizer.zero_grad()
            outputs = model(batch['input_num'],attention_mask=batch['input_attention'],target=batch['target_ids'])
            outputs[0].backward()
            adams_optimizer.step()

    generate_song(model, "I love deep learning")